## STOCKHOLM TEAM

## Exploratory Data Analysis of the Indian StartUp Funding Ecosystem 

### Business Understanding

**Project Description:**

Explore the Indian startup funding ecosystem through an in-depth analysis of funding data from 2019 to 2021. Gain insights into key trends, funding patterns, and factors driving startup success. Investigate the relationship between funding and startup growth, with a focus on temporal patterns and city-level dynamics. Identify preferred sectors for investment and uncover industry-specific funding trends. This exploratory data analysis provides a comprehensive overview of the Indian startup ecosystem, offering valuable insights for entrepreneurs, investors, and policymakers.

## Data Understanding

This project aims to explore and gain a deeper understanding of the Indian startup funding ecosystem. The dataset used for analysis contains information about startup funding from 2019 to 2021. The dataset includes various attributes such as the company's name, sector, funding amount, funding round, investor details, and location.

To conduct a comprehensive analysis, we will examine the dataset to understand its structure, contents, and any potential data quality issues. By understanding the data, we can ensure the accuracy and reliability of our analysis.

The key attributes in the dataset include:

- **Company**: The name of the startup receiving funding.
- **Sector**: The industry or sector to which the startup belongs.
- **Amount**: The amount of funding received by the startup.
- **Stage**: The round of funding (e.g., seed, series A, series B).
- **Location**: The city or region where the startup is based.
- **About**: What the company does.
- **Funding Year**:When the company was funded

By examining these attributes, we can uncover insights about the funding landscape, identify trends in funding amounts and rounds, explore the preferred sectors for investment, and analyze the role of cities in the startup ecosystem.

Throughout the analysis, we will use visualizations and statistical techniques to present the findings effectively. By understanding the data and its characteristics, we can proceed with confidence in our analysis, derive meaningful insights, and make informed decisions based on the findings.

### Hypothesis:

#### NULL Hypothesis (HO) :

#### **The sector of a company does not have an impact on the amount of funding it receives.**


#### ALTERNATE Hypothesis (HA):

#### **The sector of a company does have an impact on the amount of funding it receives.**




##  Research / Analysis Questions:

1. What are the most common industries represented in the datasets?

2. How does the funding amount vary across different rounds/series in the datasets?
   
3. Which locations have the highest number of companies in the datasets?
   
4. What kind of investment type should startups look for depending on their industry type? (EDA: Analysis of funding preferences by industry)

5. Are there any correlations between the funding amount and the company's sector or location?
   
6. What are the top investors in the datasets based on the number of investments made?
   
7. Which industries are favored by investors based on the number of funding rounds? (EDA: Top 10 industries which are favored by investors)

8. Are there any outliers in the funding amounts in the datasets?
   
9.  Is there a relationship between the company's sector and the presence of certain investors?
    
10. What is the range of funds generally received by startups in India (Max, min, avg, and count of funding)? (EDA: Descriptive statistics of funding amounts)


## Data Preparation

Before diving into the analysis, we will preprocess and clean the data to ensure its quality and suitability for analysis. This may involve handling missing values, correcting data types, and addressing any inconsistencies or outliers that could affect the accuracy of our results.

Once the data is prepared, we will be ready to perform an in-depth exploratory analysis of the Indian startup funding ecosystem. The analysis will involve answering specific research questions, identifying patterns and trends, and generating meaningful visualizations to present the findings.

Through this process of data understanding and preparation, we will set a solid foundation for conducting a robust and insightful analysis of the Indian startup funding data.

**The data for each year is sourced from separate two csv files and two from a remote server. They will be merged later to one dataset**

### Load the Packages/Modules

In [ ]:
%pip install forex-python
%pip install pandas
%pip install python-dotenv
%pip install seaborn
%pip install matplotlib
%pip install pyodbc
%pip install numpy

In [260]:
# Importing the Modules needed
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import pyodbc #just installed with pip
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import warnings 
warnings.filterwarnings('ignore')

from forex_python.converter import CurrencyRates
import re 

### Import Datasets

In [309]:
df = pd.read_csv('startup_funding2018.csv') # read the data_2018 and convert it to pandas data frame 

In [263]:
df2 = pd.read_csv('startup_funding2019.csv') # read the data_2019 and convert it to pandas data frame

#### Accessing the Remote Server Datasets

In [264]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [265]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)

In [266]:
# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 
query1 = "SELECT * FROM dbo.LP1_startup_funding2020"
query2 = "SELECT * FROM dbo.LP1_startup_funding2021"
df3 = pd.read_sql(query1, connection)
df4 = pd.read_sql(query2, connection)

In [267]:
#Set display options to show all values without truncation
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### 2018 Data

In [268]:
df.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, Sponsorship, Ticketing",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and frolic of Colleges."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy farmers in rural Maharashtra.
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organization which offers short term financial suport to corporate employees.
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants to create a difference by selling globally.


In [269]:
df.shape

(526, 6)

In [270]:
df.columns

Index(['Company Name', 'Industry', 'Round/Series', 'Amount', 'Location',
       'About Company'],
      dtype='object')

In [271]:
df.info()  # Get information about the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB


In [273]:
df.describe(include='object')  # Generate descriptive statistics of the DataFrame

,Company Name,Industry,Round/Series,Amount,Location,About Company
count,526,526,526,526,526,526
unique,525,405,21,198,50,524
top,TheCollegeFever,—,Seed,—,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and frolic of Colleges."
freq,2,30,280,148,102,2


now we have some description about the data set, we can now move on with data cleaning
 
MISSING VALUES 

In [275]:
missing_values = df.isnull().sum() # looking for missing values 

print(missing_values)

Company Name     0
Industry         0
Round/Series     0
Amount           0
Location         0
About Company    0
dtype: int64


#### Handling Duplicated Data

In [332]:
columns_to_check = ['Company Name', 'Industry', 'Round/Series', 'Amount', 'Location', 'About Company']

for column in columns_to_check:
    has_duplicates = df[column].duplicated().any()
    print(f'{column}: {has_duplicates}')

Company Name: True
Industry: True
Round/Series: True
Amount: True
Location: True
About Company: True


In [333]:
df.drop_duplicates(subset=['Company Name', 'Industry', 'Round/Series', 'Amount', 'Location', 'About Company'], inplace=True)

Standardizing Data Formats

now let's see how we can standardize tha data set to make sure we have the same format of data points 

first let's check for dash symbols within the columns using a simple python function 

In [334]:
columns_to_check = ['Amount', 'Company Name', 'Location', 'About Company', 'Industry', 'Round/Series']

for column in columns_to_check:
    has_dash_symbols = df[column].str.contains('—').any()
    print(f"{column}: {has_dash_symbols}")

Amount: True
Company Name: False
Location: False
About Company: False
Industry: True
Round/Series: False


now let's handle the dash symbols in **the Amount column**, clean and format the amount the column correctly & Convert Currency to USD

In [335]:
df['Amount'].head()

0         250000
1    ₹40,000,000
2    ₹65,000,000
3        2000000
4              —
Name: Amount, dtype: object

In [336]:
# Cleaning the Amounts column
df['Amount'] = df['Amount'].apply(str)
df['Amount'].replace(",", "", inplace = True, regex=True)
df['Amount'].replace("—", 0, inplace = True, regex=True)
df['Amount'].replace("$", "", inplace = True, regex=True)

In [337]:
# Create an instance of CurrencyRates
c = CurrencyRates()

In [338]:
# Creating temporary columns to help with the conversion of INR to USD
df['Indiancurr'] = df['Amount'].str.rsplit('₹', n=2).str[1]
df['Indiancurr'] = df['Indiancurr'].apply(float).fillna(0)
df['UsCurr'] = df['Indiancurr'] * c.get_rate('INR', 'USD')
df['UsCurr'] = df['UsCurr'].replace(0, np.nan)
df['UsCurr'] = df['UsCurr'].fillna(df['Amount'])
df['UsCurr'] = df['UsCurr'].replace("$", "", regex=True)
df['Amount'] = df['UsCurr']
df['Amount'] = df['Amount'].apply(lambda x: float(str(x).replace("$","")))
df['Amount'] = df['Amount'].replace(0, np.nan)

# Define a lambda function to format the amount
format_amount = lambda amount: "{:,.2f}".format(amount)

# Apply the formatting lambda function to the 'Amount' column
df['Amount'] = df['Amount'].map(format_amount)


In [339]:
df['Amount'].head()

0      250,000.00
1      485,093.94
2      788,277.65
3    2,000,000.00
4             nan
Name: Amount, dtype: object

In [340]:
df['Amount'] = df['Amount'].str.replace(',', '').astype(float)
type(df['Amount'][0])

numpy.float64

#### Handling Categorical Data
NOW LET'S 

handle the categorical data in the 'Industry', 'Round/Series', and 'Location' columns

Analyzing unique values
Start by examining the unique values in each column to identify any inconsistencies or variations we do this 
Using the unique() function to get the unique values in each column.

### Location Column

In [341]:
df['Location'].unique() # 

array(['Bengaluru', 'Mumbai', 'Gurgaon', 'Noida', 'Hyderabad', 'Kalkaji',
       'Delhi', 'India', 'Hubli', 'Chennai', 'Mohali', 'Kolkata', 'Pune',
       'Jodhpur', 'Kanpur', 'Ahmedabad', 'Azadpur', 'Haryana', 'Cochin',
       'Faridabad', 'Jaipur', 'Kota', 'Anand', 'Thane', 'Margão',
       'Indore', 'Alwar', 'Kannur', 'Trivandrum', 'Ernakulam',
       'Kormangala', 'Uttar Pradesh', 'Andheri', 'Mylapore', 'Ghaziabad',
       'Kochi', 'Powai', 'Guntur', 'Kalpakkam', 'Bhopal', 'Coimbatore',
       'Worli', 'Alleppey', 'Chandigarh', 'Guindy', 'Lucknow'],
      dtype=object)

In [311]:
df['Location'].value_counts()

Bangalore, Karnataka, India         102
Mumbai, Maharashtra, India           94
Bengaluru, Karnataka, India          55
Gurgaon, Haryana, India              52
New Delhi, Delhi, India              51
Pune, Maharashtra, India             20
Chennai, Tamil Nadu, India           19
Hyderabad, Andhra Pradesh, India     18
Delhi, Delhi, India                  16
Noida, Uttar Pradesh, India          15
Haryana, Haryana, India              11
Jaipur, Rajasthan, India              9
Ahmedabad, Gujarat, India             6
Kolkata, West Bengal, India           6
Bangalore City, Karnataka, India      5
Indore, Madhya Pradesh, India         4
India, Asia                           4
Kormangala, Karnataka, India          3
Ghaziabad, Uttar Pradesh, India       2
Kochi, Kerala, India                  2
Bhopal, Madhya Pradesh, India         2
Thane, Maharashtra, India             2
Jodhpur, Rajasthan, India             1
Powai, Assam, India                   1
Andheri, Maharashtra, India           1


In [315]:
# The 'Location' column is in the format, 'City, Region, Country',
# Only 'City' aspect is needed for this analysis
# Take all character until we reach the first comma sign

df['Location'] = df['Location'].apply(str)
df['Location'] = df['Location'].str.split(',').str[0]
df['Location'] = df['Location'].replace("'","",regex=True)

In [316]:
# From obersavtion, there is use of official and unofficial names of certain cities.
# The incorrect names need to be rectified for correct analysis, eg A city with more than one name.
df["Location"] = df["Location"].replace (['Bangalore','Bangalore City','Belgaum'], 'Bengaluru')
df['Location'] = df['Location'].str.replace('New Delhi','Delhi')

In [317]:
df['Location']

0          Bengaluru
1             Mumbai
2            Gurgaon
3              Noida
4          Hyderabad
5          Bengaluru
6            Kalkaji
7          Hyderabad
8             Mumbai
9          Bengaluru
10             Delhi
11         Bengaluru
12             India
13             Hubli
14         Bengaluru
15         Bengaluru
16            Mumbai
17         Bengaluru
18             Delhi
19           Chennai
20            Mumbai
21            Mumbai
22             Delhi
23             Delhi
24         Bengaluru
25            Mohali
26           Chennai
27            Mumbai
28            Mumbai
29         Hyderabad
30             Delhi
31           Kolkata
32         Bengaluru
33         Bengaluru
34            Mumbai
35         Bengaluru
36            Mumbai
37             Delhi
38            Mumbai
39           Chennai
40             Delhi
41         Hyderabad
42             India
43            Mumbai
44             Delhi
45             Delhi
46         Bengaluru
47           

In [318]:
df['Location'].unique() 

array(['Bengaluru', 'Mumbai', 'Gurgaon', 'Noida', 'Hyderabad', 'Kalkaji',
       'Delhi', 'India', 'Hubli', 'Chennai', 'Mohali', 'Kolkata', 'Pune',
       'Jodhpur', 'Kanpur', 'Ahmedabad', 'Azadpur', 'Haryana', 'Cochin',
       'Faridabad', 'Jaipur', 'Kota', 'Anand', 'Thane', 'Margão',
       'Indore', 'Alwar', 'Kannur', 'Trivandrum', 'Ernakulam',
       'Kormangala', 'Uttar Pradesh', 'Andheri', 'Mylapore', 'Ghaziabad',
       'Kochi', 'Powai', 'Guntur', 'Kalpakkam', 'Bhopal', 'Coimbatore',
       'Worli', 'Alleppey', 'Chandigarh', 'Guindy', 'Lucknow'],
      dtype=object)

In [319]:
df['Location'].value_counts()

Bengaluru        163
Mumbai            94
Delhi             67
Gurgaon           52
Pune              20
Chennai           19
Hyderabad         18
Noida             15
Haryana           11
Jaipur             9
Kolkata            6
Ahmedabad          6
India              4
Indore             4
Kormangala         3
Bhopal             2
Kochi              2
Ghaziabad          2
Thane              2
Faridabad          1
Mylapore           1
Guindy             1
Chandigarh         1
Alleppey           1
Worli              1
Coimbatore         1
Kalkaji            1
Kalpakkam          1
Guntur             1
Powai              1
Hubli              1
Mohali             1
Andheri            1
Cochin             1
Uttar Pradesh      1
Jodhpur            1
Ernakulam          1
Trivandrum         1
Kannur             1
Alwar              1
Kanpur             1
Margão             1
Azadpur            1
Anand              1
Kota               1
Lucknow            1
Name: Location, dtype: int64

### Industry Column

In [320]:
df['Industry']

0                                                                           Brand Marketing, Event Promotion, Marketing, Sponsorship, Ticketing
1                                                                                                                          Agriculture, Farming
2                                                                                              Credit, Financial Services, Lending, Marketplace
3                                                                                                                   Financial Services, FinTech
4                                                                                                            E-Commerce Platforms, Retail, SaaS
5                                                                                                              Cloud Infrastructure, PaaS, SaaS
6                                                                                                                Internet, Leisure, Mark

In [321]:
# let's check all the unique values in the industry column
df['Industry'].unique()

array(['Brand Marketing, Event Promotion, Marketing, Sponsorship, Ticketing',
       'Agriculture, Farming',
       'Credit, Financial Services, Lending, Marketplace',
       'Financial Services, FinTech',
       'E-Commerce Platforms, Retail, SaaS',
       'Cloud Infrastructure, PaaS, SaaS',
       'Internet, Leisure, Marketplace', 'Market Research',
       'Information Services, Information Technology', 'Mobile Payments',
       'B2B, Shoes', 'Internet',
       'Apps, Collaboration, Developer Platform, Enterprise Software, Messaging, Productivity Tools, Video Chat',
       'Food Delivery', 'Industrial Automation',
       'Automotive, Search Engine, Service Industry',
       'Finance, Internet, Travel',
       'Accounting, Business Information Systems, Business Travel, Finance, SaaS',
       'Artificial Intelligence, Product Search, SaaS, Service Industry, Software',
       'Internet of Things, Waste Management',
       'Air Transportation, Freight Service, Logistics, Marine Transport

In [322]:
df['Industry'].value_counts()

—                                                                                                                                           30
Financial Services                                                                                                                          15
Education                                                                                                                                    8
Information Technology                                                                                                                       7
Health Care, Hospital                                                                                                                        5
Finance, Financial Services                                                                                                                  5
Fitness, Health Care, Wellness                                                                                                               4

BELOW WE WANT TO HANDLE, Title casing, leading and trailing spaces and also standardize the indusrty column 

In [257]:
# Get unique values in the 'Industry' column
unique_values = df['Industry'].unique()
# Create a set to store the delimiters
delimiters = set()

# Iterate over the unique values
for value in unique_values:
    parts = re.split(',|;|/|-', value) # Split the value by commas and other delimiters
    delimiters.update(filter(lambda x: x != '', parts[1:])) # Add the delimiters to the set
# Print the identified delimiters
print(delimiters)

{' Railroad', ' Marketplace', ' Fraud Detection', ' Navigation', ' Enterprise', ' Video on Demand', ' Finance', ' Energy Storage', ' Mechanical Design', ' Consumer', ' Medical Device', ' Speech Recognition', ' Property Management', ' Tea', " Men's", ' Drones', ' Analytics', ' Industrial Automation', ' Higher Education', ' Diabetes', ' Secondary Education', ' Health Insurance', ' Virtual Assistant', ' Marine Transportation', ' Real Estate', ' Leisure', ' mHealth', ' Information Technology', 'Learning', ' Virtual Reality', ' Last Mile Transportation', ' Battery', ' Cricket', ' Developer Tools', ' Network Hardware', ' Facilities Support Services', ' Social Entrepreneurship', ' Home Renovation', ' InsurTech', ' Financial Services', ' Mobile', ' Social Media', ' Financial Exchanges', ' Online Portals', ' Tutoring', ' Training', ' Knowledge Management', " Women's", ' Electronics', ' Business Travel', ' Home Improvement', ' Transportation', ' Beauty', ' Manufacturing', ' Home Decor', ' Shippi

In [324]:
# keeping only the first unique vlaues in the Industry column
df['Industry'] = df['Industry'].str.split(',').str[0]

In [325]:
#converting the industry names in the column to title case
df['Industry'] = df['Industry'].str.title()

In [326]:
# checking if there are any leading or trailing spaces in the industry names in the 'Industry' column
has_spaces = df['Industry'].str.contains('^s|s$', regex=True)

rows_with_spaces = df[has_spaces]
print(rows_with_spaces)

                                       Company Name  \
3                                       PayMe India   
4                                          Eunimart   
8                                      Freightwalla   
9                                Microchip Payments   
12                                            Flock   
19                 Antariksh Waste Ventures Pvt ltd   
21                                       PaisaDukan   
23                                         BlueJack   
26                                            Pando   
27                                          Mintifi   
30                                     Chariot Tech   
32                                          Loanzen   
39                                       Kaleidofin   
44                                        Letstrack   
46                                            Finzy   
47                                       Fitternity   
59                                           Rooter   
65        

In [328]:
# remove the leading or trailing spaces from the industry names in the 'Industry' column
df['Industry'] = df['Industry'].str.strip()

In [330]:
# Replace '-' with NaN
df['Industry'].replace('-', np.nan, inplace=True)

In [342]:
df.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company,Indiancurr,UsCurr
0,TheCollegeFever,Brand Marketing,Seed,250000.00,Bengaluru,"TheCollegeFever is a hub for fun, fiesta and frolic of Colleges.",0.0,250000
1,Happy Cow Dairy,Agriculture,Seed,485093.94,Mumbai,A startup which aggregates milk from dairy farmers in rural Maharashtra.,40000000.0,485093.936326
2,MyLoanCare,Credit,Series A,788277.65,Gurgaon,Leading Online Loans Marketplace in India,65000000.0,788277.646529
3,PayMe India,Financial Services,Angel,2000000.00,Noida,PayMe India is an innovative FinTech organization which offers short term financial suport to corporate employees.,0.0,2000000
4,Eunimart,E-Commerce Platforms,Seed,NaN,Hyderabad,Eunimart is a one stop solution for merchants to create a difference by selling globally.,0.0,0


### Round/Series Column

In [343]:
df['Round/Series'].unique()

array(['Seed', 'Series A', 'Angel', 'Series B', 'Pre-Seed',
       'Private Equity', 'Venture - Series Unknown', 'Grant',
       'Debt Financing', 'Post-IPO Debt', 'Series H', 'Series C',
       'Series E', 'Corporate Round', 'Undisclosed',
       'https://docs.google.com/spreadsheets/d/1x9ziNeaz6auNChIHnMI8U6kS7knTr3byy_YBGfQaoUA/edit#gid=1861303593',
       'Series D', 'Secondary Market', 'Post-IPO Equity',
       'Non-equity Assistance', 'Funding Round'], dtype=object)

In [344]:
df['Round/Series'].value_counts()

Seed                                                                                                       279
Series A                                                                                                    73
Angel                                                                                                       37
Venture - Series Unknown                                                                                    37
Series B                                                                                                    20
Series C                                                                                                    16
Debt Financing                                                                                              13
Private Equity                                                                                              10
Corporate Round                                                                                              8
P

In [345]:
df['Round/Series']=df['Round/Series'].replace('Undisclosed',np.nan)
df['Round/Series']=df['Round/Series'].replace('Venture - Series Unknown',np.nan)
df['Round/Series'] = df['Round/Series'].replace('https://docs.google.com/spreadsheets/d/1x9ziNeaz6auNChIHnMI8U6kS7knTr3byy_YBGfQaoUA/edit#gid=1861303593', 'nan')
df['Round/Series'] = df['Round/Series'].replace('nan', np.nan)

In [346]:
df['Round/Series'].unique()

array(['Seed', 'Series A', 'Angel', 'Series B', 'Pre-Seed',
       'Private Equity', nan, 'Grant', 'Debt Financing', 'Post-IPO Debt',
       'Series H', 'Series C', 'Series E', 'Corporate Round', 'Series D',
       'Secondary Market', 'Post-IPO Equity', 'Non-equity Assistance',
       'Funding Round'], dtype=object)

In [347]:
df['Round/Series'].value_counts()

Seed                     279
Series A                  73
Angel                     37
Series B                  20
Series C                  16
Debt Financing            13
Private Equity            10
Corporate Round            8
Pre-Seed                   6
Series E                   5
Grant                      4
Series D                   3
Secondary Market           3
Post-IPO Equity            3
Post-IPO Debt              2
Non-equity Assistance      1
Series H                   1
Funding Round              1
Name: Round/Series, dtype: int64

In [348]:
df.columns

Index(['Company Name', 'Industry', 'Round/Series', 'Amount', 'Location',
       'About Company', 'Indiancurr', 'UsCurr'],
      dtype='object')

In [349]:
df.drop(columns=['Indiancurr','UsCurr'], inplace=True)

In [350]:
df.insert(6,"Funding Year", 2018)

In [351]:
df.rename(columns = {'Company Name':'Company',
                        'Industry':'Sector',
                        'Amount':'Amount',
                        'About Company':'About',
                        'Round/Series' : 'Stage'},
             inplace = True)

In [352]:
df.head()

,Company,Sector,Stage,Amount,Location,About,Funding Year
0,TheCollegeFever,Brand Marketing,Seed,250000.00,Bengaluru,"TheCollegeFever is a hub for fun, fiesta and frolic of Colleges.",2018
1,Happy Cow Dairy,Agriculture,Seed,485093.94,Mumbai,A startup which aggregates milk from dairy farmers in rural Maharashtra.,2018
2,MyLoanCare,Credit,Series A,788277.65,Gurgaon,Leading Online Loans Marketplace in India,2018
3,PayMe India,Financial Services,Angel,2000000.00,Noida,PayMe India is an innovative FinTech organization which offers short term financial suport to corporate employees.,2018
4,Eunimart,E-Commerce Platforms,Seed,NaN,Hyderabad,Eunimart is a one stop solution for merchants to create a difference by selling globally.,2018


In [ ]:
df.to_csv('df18.csv', index=False)

#### 2019 Data

In [353]:
df2.head()

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,"A learning platform that provides topic-based journey, animated videos, quizzes, infographic and mock tests to students","Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ Labs","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, processing and marketing of seeds",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN


In [354]:
df2.shape

(89, 9)

In [355]:
df2.columns

Index(['Company/Brand', 'Founded', 'HeadQuarter', 'Sector', 'What it does',
       'Founders', 'Investor', 'Amount($)', 'Stage'],
      dtype='object')

In [ ]:
df2.info() # Getting inforamation about the data2 dataframe

In [356]:
df2.describe(include='object') # getting General descriptive statistics of the data2 dataFrame

,Company/Brand,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
count,89,70,84,89,86,89,89,43
unique,87,17,52,88,85,86,50,15
top,Kratikal,Bangalore,Edtech,Online meat shop,"Vivek Gupta, Abhay Hanjura",Undisclosed,Undisclosed,Series A
freq,2,21,7,2,2,3,12,10


#### Handling Duplicated Data

In [357]:
columns_to_check2 = ['Company/Brand', 'Founded', 'HeadQuarter', 'Sector', 'What it does', 'Founders', 'Investor', 'Amount($)', 'Stage',]

for column2 in columns_to_check2:
    has_duplicates2 = df2[column2].duplicated().any()
    print(f'{column2}: {has_duplicates2}')

Company/Brand: True
Founded: True
HeadQuarter: True
Sector: True
What it does: True
Founders: True
Investor: True
Amount($): True
Stage: True


In [358]:
df2.drop_duplicates(subset=['Company/Brand', 'Founded', 'HeadQuarter', 'Sector', 'What it does', 'Founders', 'Investor', 'Amount($)', 'Stage',], inplace=True)

now we have some description about the data set, we can now move on with data cleaning
 
MISSING VALUES 

In [359]:
missing_values2 = df2.isnull().sum() # looking for missing values in dataFrame 2
missing_values2

Company/Brand     0
Founded          29
HeadQuarter      19
Sector            5
What it does      0
Founders          3
Investor          0
Amount($)         0
Stage            46
dtype: int64

LET'S DEAL WITH THE MISSING VALUES FROM THE ABOVE OUTPUT

In [360]:
df2['HeadQuarter'].unique()

array([nan, 'Mumbai', 'Chennai', 'Telangana', 'Pune', 'Bangalore',
       'Noida', 'Delhi', 'Ahmedabad', 'Gurugram', 'Haryana', 'Chandigarh',
       'Jaipur', 'New Delhi', 'Surat', 'Uttar pradesh', 'Hyderabad',
       'Rajasthan'], dtype=object)

In [361]:
df2['HeadQuarter']= df2['HeadQuarter'].str.replace('New Delhi','Delhi')
df2["HeadQuarter"] = df2["HeadQuarter"].replace (['Bangalore','Bangalore City','Belgaum'], 'Bengaluru')

In [362]:
df2['HeadQuarter'].unique()

array([nan, 'Mumbai', 'Chennai', 'Telangana', 'Pune', 'Bengaluru',
       'Noida', 'Delhi', 'Ahmedabad', 'Gurugram', 'Haryana', 'Chandigarh',
       'Jaipur', 'Surat', 'Uttar pradesh', 'Hyderabad', 'Rajasthan'],
      dtype=object)

In [363]:
df2['Sector'].unique()

array(['Ecommerce', 'Edtech', 'Interior design', 'AgriTech', 'Technology',
       'SaaS', 'AI & Tech', 'E-commerce', 'E-commerce & AR', 'Fintech',
       'HR tech', 'Food tech', 'Health', 'Healthcare', 'Safety tech',
       'Pharmaceutical', 'Insurance technology', 'AI', 'Foodtech', 'Food',
       'IoT', 'E-marketplace', 'Robotics & AI', 'Logistics', 'Travel',
       'Manufacturing', 'Food & Nutrition', 'Social Media', nan,
       'E-Sports', 'Cosmetics', 'B2B', 'Jewellery', 'B2B Supply Chain',
       'Games', 'Food & tech', 'Accomodation', 'Automotive tech',
       'Legal tech', 'Mutual Funds', 'Cybersecurity', 'Automobile',
       'Sports', 'Healthtech', 'Yoga & wellness', 'Virtual Banking',
       'Transportation', 'Transport & Rentals',
       'Marketing & Customer loyalty', 'Infratech', 'Hospitality',
       'Automobile & Technology', 'Banking'], dtype=object)

In [365]:
df2['Stage'].unique()

array([nan, 'Series C', 'Fresh funding', 'Series D', 'Pre series A',
       'Series A', 'Series G', 'Series B', 'Post series A',
       'Seed funding', 'Seed fund', 'Series E', 'Series F', 'Series B+',
       'Seed round', 'Pre-series A'], dtype=object)

In [366]:
# Replacing specific values with NaN
df2['HeadQuarter'].replace('Not available', np.nan, inplace=True)

# Replacing empty strings with NaN
df2['Sector'].replace('', np.nan, inplace=True)
df2['Stage'].replace('', np.nan, inplace=True)

In [367]:
df2.isnull().sum() # let's check for null vlaues and sum them up 

Company/Brand     0
Founded          29
HeadQuarter      19
Sector            5
What it does      0
Founders          3
Investor          0
Amount($)         0
Stage            46
dtype: int64

Standardizing Data Formats

now let's see how we can standardize tha data set to make sure we have the same format of data points 

first let's check for dash symbols within the columns using a simple python function 

In [368]:
# checking for '-' symbol within the columns

columns_to_check2 = ['Company/Brand', 'HeadQuarter', 'Sector', 'What it does', 'Amount($)', 'Stage']

for column2 in columns_to_check2:
    has_dash_symbols2 = df2[column2].astype(str).str.contains('-').any()
    print(f'{column2}: {has_dash_symbols2}')

Company/Brand: False
HeadQuarter: False
Sector: True
What it does: True
Amount($): False
Stage: True


In [369]:
# checking for currency symbol 

columns_to_check2 = ['Company/Brand','HeadQuarter', 'Sector', 'What it does', 'Amount($)']

for column2 in columns_to_check2:
    has_currency_symbols = df2[column2].astype(str).str.contains('[$₹]').any()
    print(f'{column2}: {has_currency_symbols}')

Company/Brand: False
HeadQuarter: False
Sector: False
What it does: False
Amount($): True


In [370]:
# replacing the '-' symbols using a simple function 

dash_currency_columns = ['Sector', 'What it does', 'Stage']

for dash_columns2 in dash_currency_columns:
    dash_replaced2 = df2[dash_columns2].replace('-', np.nan, inplace=True)

now let's handle the dash symbols in the Amount column, clean and format the amount the column correctly 

In [371]:
df2['Amount($)'].unique() # let's check for unique values 

array(['$6,300,000', '$150,000,000', '$28,000,000', '$30,000,000',
       '$6,000,000', 'Undisclosed', '$1,000,000', '$20,000,000',
       '$275,000,000', '$22,000,000', '$5,000,000', '$140,500',
       '$540,000,000', '$15,000,000', '$182,700', '$12,000,000',
       '$11,000,000', '$15,500,000', '$1,500,000', '$5,500,000',
       '$2,500,000', '$140,000', '$230,000,000', '$49,400,000',
       '$32,000,000', '$26,000,000', '$150,000', '$400,000', '$2,000,000',
       '$100,000,000', '$8,000,000', '$100,000', '$50,000,000',
       '$120,000,000', '$4,000,000', '$6,800,000', '$36,000,000',
       '$5,700,000', '$25,000,000', '$600,000', '$70,000,000',
       '$60,000,000', '$220,000', '$2,800,000', '$2,100,000',
       '$7,000,000', '$311,000,000', '$4,800,000', '$693,000,000',
       '$33,000,000'], dtype=object)

In [372]:
# Cleaning the Amounts column & # removing the currency symbol in df_2019
df2['Amount($)'] = df2['Amount($)'].astype(str).str.replace('[\₹$,]', '', regex=True)
df2['Amount($)'] = df2['Amount($)'].str.replace('Undisclosed', '0', regex=True)
df2['Amount($)'].replace(",", "", inplace = True, regex=True)
df2['Amount($)'].replace("—", 0, inplace = True, regex=True)

In [373]:
df2['Amount($)'] = df2['Amount($)'].astype(float) # here we are converting the amount column to float data type 
type(df2['Amount($)'][0])

numpy.float64

In [374]:
df2['Amount($)'] # here we are looking at the Amount column 

0       6300000.0
1     150000000.0
2      28000000.0
3      30000000.0
4       6000000.0
5             0.0
6             0.0
7       1000000.0
8      20000000.0
9     275000000.0
10            0.0
11     22000000.0
12      5000000.0
13       140500.0
14            0.0
15      5000000.0
16    540000000.0
17     15000000.0
18       182700.0
19            0.0
20      5000000.0
21     12000000.0
22     11000000.0
23            0.0
24     15500000.0
25      1500000.0
26      5500000.0
27      5000000.0
28     12000000.0
29      2500000.0
30     30000000.0
31       140000.0
32            0.0
33    230000000.0
34     20000000.0
35     49400000.0
36     32000000.0
37     26000000.0
38       150000.0
39       400000.0
40      2000000.0
41    100000000.0
42      8000000.0
43      1500000.0
44       100000.0
45            0.0
46     50000000.0
47      6000000.0
48    120000000.0
49      4000000.0
50     30000000.0
51      4000000.0
52      1500000.0
53      1000000.0
54            0.0
55        

In [375]:
df2['Amount($)'].unique() # this line of code looks at the unique value 

array([6.300e+06, 1.500e+08, 2.800e+07, 3.000e+07, 6.000e+06, 0.000e+00,
       1.000e+06, 2.000e+07, 2.750e+08, 2.200e+07, 5.000e+06, 1.405e+05,
       5.400e+08, 1.500e+07, 1.827e+05, 1.200e+07, 1.100e+07, 1.550e+07,
       1.500e+06, 5.500e+06, 2.500e+06, 1.400e+05, 2.300e+08, 4.940e+07,
       3.200e+07, 2.600e+07, 1.500e+05, 4.000e+05, 2.000e+06, 1.000e+08,
       8.000e+06, 1.000e+05, 5.000e+07, 1.200e+08, 4.000e+06, 6.800e+06,
       3.600e+07, 5.700e+06, 2.500e+07, 6.000e+05, 7.000e+07, 6.000e+07,
       2.200e+05, 2.800e+06, 2.100e+06, 7.000e+06, 3.110e+08, 4.800e+06,
       6.930e+08, 3.300e+07])

In [376]:
# Dropping the columns that are not important to our analysis

df2.drop(columns=['Founded','Founders','Investor'], inplace=True)

In [377]:
df2.insert(6,"Funding Year", 2019)

In [378]:
df2.rename(columns = {'Company/Brand':'Company',
                        'HeadQuarter':'Location',
                        'Amount($)':'Amount',
                        'What it does':'About'},
             inplace = True)

In [379]:
df2.head()

,Company,Location,Sector,About,Amount,Stage,Funding Year
0,Bombay Shaving,NaN,Ecommerce,Provides a range of male grooming products,6300000.0,NaN,2019
1,Ruangguru,Mumbai,Edtech,"A learning platform that provides topic-based journey, animated videos, quizzes, infographic and mock tests to students",150000000.0,Series C,2019
2,Eduisfun,Mumbai,Edtech,It aims to make learning fun via games.,28000000.0,Fresh funding,2019
3,HomeLane,Chennai,Interior design,Provides interior designing solutions,30000000.0,Series D,2019
4,Nu Genes,Telangana,AgriTech,"It is a seed company engaged in production, processing and marketing of seeds",6000000.0,NaN,2019


In [380]:
df2.to_csv('df_19.csv', index=False)

### 2020 Data

In [ ]:
df3.head()

#### 2021 Data

In [ ]:
df4.head()

In [ ]:
df4.shape

In [ ]:
df4.columns

In [ ]:
df4.info()

In [ ]:
df4.describe(include='object')

In [ ]:
missing_values4 = df4.isnull().sum() # looking for missing values in dataFrame 4
missing_values4

#### Handling Duplicated Data

In [ ]:
columns_to_check2 = ['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does', 'Founders', 'Investor', 'Amount', 'Stage']

for column2 in columns_to_check2:
    has_duplicates2 = df4[column2].duplicated().any()
    print(f'{column2}: {has_duplicates2}')

In [ ]:
df4.drop_duplicates(subset=['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does', 'Founders', 'Investor', 'Amount', 'Stage'], inplace=True)

### Location Column

In [ ]:
df4['HeadQuarter'].unique()

In [ ]:
# From obersavtion, there is use of official and unofficial names of certain cities.
# The incorrect names need to be rectified for correct analysis, eg A city with more than one name.
df4['HeadQuarter'] = df4['HeadQuarter'].replace (['Bangalore','Bangalore City','Belgaum'], 'Bengaluru')
df4['HeadQuarter'].replace('Gurugram\t#REF!','Gurugram',inplace =True, regex=True)
df4['HeadQuarter'] = df4['HeadQuarter'].str.replace('New Delhi','Delhi')

In [ ]:
df4[df4['HeadQuarter'].isin(['Online Media\t#REF!', 'Pharmaceuticals\t#REF!','Computer Games','Information Technology & Services','Food & Beverages'])]

In [ ]:
df4.loc[df4["Company_Brand"] == "FanPlay", ["HeadQuarter", "Amount", "Stage"]] = ["None", "$1200000","None"]
df4.loc[df4["Company_Brand"] == "FanPlay"]

In [ ]:
df4.loc[df4["Company_Brand"] == "MasterChow", ["HeadQuarter", "Sector"]] = ["Hauz Khas", "Food & Beverages"]
df4.loc[df4["Company_Brand"] == "MasterChow"]

In [ ]:
df4.loc[df4["Company_Brand"] == "Fullife Healthcare", ["HeadQuarter","Sector","What_it_does","Investor", "Amount", "Stage"]] = ["None","Pharmaceuticals","Primary Business is Development and Manufactur...","Varun Khanna", "$22000000","Series C"]
df4.loc[df4["Company_Brand"] == "Fullife Healthcare"]

In [ ]:

df4.loc[df4["Company_Brand"] == "Peak", ["HeadQuarter", "Sector"]] = ["Manchester", "Information Technology & Services"]
df4.loc[df4["Company_Brand"] == "Peak"]

In [ ]:

df4.loc[df4["Company_Brand"] == "Sochcast", ["HeadQuarter", "Sector",'What_it_does','Founders','Investor',"Amount"]] = [np.nan, 'Online Media','Sochcast is an Audio experiences company that give the listener and creators an Immersive Audio experience','CA Harvinderjit Singh Bhatia, Garima Surana','Vinners, Raj Nayak, Amritaanshu Agrawal',"$Undisclosed"]
df4.loc[df4["Company_Brand"] == "Sochcast"]

In [ ]:
df4['Sector'].unique()

In [ ]:
df4.loc[df4["Company_Brand"] == "MoEVing", ["Sector",'What_it_does','Founders','Investor','Amount','Stage']] = [
    np.nan,"MoEVing is India's only Electric Mobility focused Technology Platform with a vision to accelerate EV adoption in India.",
    'Vikash Mishra, Mragank Jain','Anshuman Maheshwary, Dr Srihari Raju Kalidindi','$5000000','Seed']
df4.loc[df4["Company_Brand"] == "MoEVing"]

In [ ]:
df4["Stage"].unique()

In [ ]:
df4[df4["Stage"]=='$6000000']

In [ ]:
df4.loc[df4["Company_Brand"] == "MYRE Capital", ["Amount", "Stage"]] = ["6000000",np.nan]
df4.loc[df4["Company_Brand"] == "MYRE Capital"]

In [ ]:
df4["Amount"].unique()

In [ ]:
df4[df4['Amount'].isin([ 'Seed','JITO Angel Network, LetsVenture','ITO Angel Network, LetsVenture','Pre-series A','ah! Ventures'])]

In [ ]:
df4.loc[df4["Company_Brand"] == "Little Leap", ["Amount", "Stage", "Investor"]] = [
    "$300000", np.nan, "ah! Ventures"]
df4.loc[df4["Investor"] == "ah! Ventures"]

In [ ]:
df4.loc[df4["Company_Brand"] == "AdmitKard", ["Amount", "Stage", "Investor"]] = [
    "$1000000", "Pre-series A",np.nan]
df4.loc[df4["Company_Brand"] == "AdmitKard"]

In [ ]:
df4.loc[df4["Company_Brand"] == "Godamwale", ["Amount", "Stage", "Investor"]] = ["$1000000", "Seed",np.nan]
df4.loc[df4["Company_Brand"] == "Godamwale"]

In [ ]:
df4.loc[df4["Company_Brand"] == "BHyve", ["Amount", "Stage"]] = ["$300000",np.nan]
df4.loc[df4["Company_Brand"] == "BHyve"]

In [ ]:
df4.loc[df4["Company_Brand"] == "Saarthi Pedagogy", ["Amount", "Stage"]] = ["$1000000",np.nan]
df4.loc[df4["Company_Brand"] == "Saarthi Pedagogy"]

In [ ]:
# Cleaning the Amounts column & # removing the currency symbol in df_2019
df4['Amount'] = df4['Amount'].astype(str).str.replace('[\₹$,]', '', regex=True)
df4['Amount'] = df4['Amount'].str.replace('Undisclosed', '0', regex=True)
df4['Amount'] = df4['Amount'].str.replace('undisclosed', '0', regex=True)
df4['Amount'] = df4['Amount'].str.replace('None', '0', regex=True)
df4['Amount'].replace(",", "", inplace = True, regex=True)
df4['Amount'].replace("—", 0, inplace = True, regex=True)
df4['Amount'].replace("", '0', inplace=True, regex=True)

In [ ]:
df4['Amount'] = df4['Amount'].astype(float)
type(df4['Amount'][0])

In [ ]:
df4['Amount'].unique()

In [ ]:
# Dropping the columns that are not important to our analysis

df4.drop(columns=['Founders','Investor','Founded'], inplace=True)

In [ ]:
df4.insert(6,"Funding Year", 2021)

In [ ]:
df4.rename(columns = {'Company_Brand':'Company',
                        'HeadQuarter':'Location',
                        'What_it_does':'About'},
             inplace = True)

In [ ]:
df4.head()